In [1]:
import sys
import os
import pandas as pd
from datetime import datetime
dateparse = lambda x: datetime.strptime(x, '%m/%d/%Y')

# Add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

from football_predictions.data.tools.rolling_averages import rolling_averages_for_window_sizes, rolling_averages
from football_predictions.data.configuration import ROLLING_AVERAGES_COLUMNS
from football_predictions.data.tools.processed import calculate_rolling_averages_home_team, calculate_rolling_averages_away_team

ROLLING_AVERAGES_WINDOW_SIZES = [2,3]

In [2]:
data = pd.read_csv('../data/interim/premier_league/premier_league_2324.csv',date_parser=dateparse)
data['Date'] = pd.to_datetime(data['Date'], dayfirst = True)
data

/var/folders/l3/bpd7p2510r9bclrfy43fqh0w0000gn/T/ipykernel_32205/1082765961.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  data = pd.read_csv('../data/interim/premier_league/premier_league_2324.csv',date_parser=dateparse)


,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,League,HomeTeamCode,AwayTeamCode,FTR_code,HTR_code
0,E0,2023-08-11,20:00,Burnley,Man City,0,3,A,0,2,...,1.97,NaN,NaN,1.92,1.95,premier_league,5,12,-1,-1
1,E0,2023-08-12,12:30,Arsenal,Nott'm Forest,2,1,H,2,0,...,1.97,2.01,2.09,1.95,1.92,premier_league,0,15,1,1
2,E0,2023-08-12,15:00,Bournemouth,West Ham,1,1,D,0,0,...,1.92,2.06,1.96,1.96,1.91,premier_league,2,18,0,0
3,E0,2023-08-12,15:00,Brighton,Luton,4,1,H,1,0,...,1.91,2.14,1.93,2.00,1.86,premier_league,4,11,1,1
4,E0,2023-08-12,15:00,Everton,Fulham,0,1,A,0,0,...,1.88,2.08,1.99,1.98,1.88,premier_league,8,9,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,E0,2024-05-19,16:00,Crystal Palace,Aston Villa,5,0,H,2,0,...,2.16,1.93,2.21,1.81,2.05,premier_league,7,1,1,1
376,E0,2024-05-19,16:00,Liverpool,Wolves,2,0,H,2,0,...,1.85,2.10,1.89,2.04,1.82,premier_league,10,19,1,1
377,E0,2024-05-19,16:00,Luton,Fulham,2,4,A,1,2,...,1.93,2.02,1.94,1.96,1.91,premier_league,11,9,-1,-1
378,E0,2024-05-19,16:00,Man City,West Ham,3,1,H,2,1,...,1.90,2.05,1.99,1.96,1.91,premier_league,12,18,1,1


In [3]:
data.dtypes

Div                     object
Date            datetime64[ns]
Time                    object
HomeTeam                object
AwayTeam                object
                     ...      
League                  object
HomeTeamCode             int64
AwayTeamCode             int64
FTR_code                 int64
HTR_code                 int64
Length: 111, dtype: object

In [4]:
data_home_grouped = data.groupby("HomeTeam")
data_away_grouped = data.groupby("AwayTeam")

## Rolling averages comparison home vs away team on one team

In [5]:
arsenal_home_group = data_home_grouped.get_group('Arsenal')
arsenal_home_group

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,League,HomeTeamCode,AwayTeamCode,FTR_code,HTR_code
1,E0,2023-08-12,12:30,Arsenal,Nott'm Forest,2,1,H,2,0,...,1.97,2.01,2.09,1.95,1.92,premier_league,0,15,1,1
21,E0,2023-08-26,15:00,Arsenal,Fulham,2,2,D,0,1,...,1.85,2.09,1.99,1.98,1.88,premier_league,0,9,0,-1
38,E0,2023-09-03,16:30,Arsenal,Man United,3,1,H,1,1,...,1.96,2.08,1.99,1.96,1.91,premier_league,0,13,1,0
54,E0,2023-09-24,14:00,Arsenal,Tottenham,2,2,D,1,1,...,1.85,2.10,1.90,2.05,1.84,premier_league,0,17,0,0
79,E0,2023-10-08,16:30,Arsenal,Man City,1,0,H,0,0,...,2.17,1.85,2.23,1.76,2.11,premier_league,0,12,1,0
92,E0,2023-10-28,15:00,Arsenal,Sheffield United,5,0,H,1,0,...,2.07,1.91,2.10,1.83,2.04,premier_league,0,16,1,1
111,E0,2023-11-11,15:00,Arsenal,Burnley,3,1,H,1,0,...,1.85,2.29,1.98,2.03,1.84,premier_league,0,5,1,1
130,E0,2023-12-02,15:00,Arsenal,Wolves,2,1,H,2,0,...,1.95,1.99,2.02,1.93,1.93,premier_league,0,19,1,1
165,E0,2023-12-17,14:00,Arsenal,Brighton,2,0,H,0,0,...,1.88,2.13,1.94,2.03,1.85,premier_league,0,4,1,0
187,E0,2023-12-28,20:15,Arsenal,West Ham,0,2,A,0,1,...,2.08,1.90,2.13,1.83,2.04,premier_league,0,18,-1,-1


In [6]:
arsenal_away_group = data_away_grouped.get_group('Arsenal')
arsenal_away_group

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,League,HomeTeamCode,AwayTeamCode,FTR_code,HTR_code
18,E0,2023-08-21,20:00,Crystal Palace,Arsenal,0,1,A,0,0,...,1.93,2.19,1.96,2.01,1.87,premier_league,7,0,-1,0
47,E0,2023-09-17,16:30,Everton,Arsenal,0,1,A,0,0,...,1.98,2.01,2.05,1.92,1.95,premier_league,8,0,-1,0
60,E0,2023-09-30,15:00,Bournemouth,Arsenal,0,4,A,0,2,...,1.89,2.08,1.93,2.02,1.86,premier_league,2,0,-1,-1
86,E0,2023-10-21,17:30,Chelsea,Arsenal,2,2,D,1,0,...,1.98,1.95,2.06,1.89,1.97,premier_league,6,0,0,1
106,E0,2023-11-04,17:30,Newcastle,Arsenal,1,0,H,0,0,...,1.93,2.07,2.00,1.97,1.91,premier_league,14,0,1,0
126,E0,2023-11-25,17:30,Brentford,Arsenal,0,1,A,0,0,...,1.83,2.17,1.87,2.11,1.78,premier_league,3,0,-1,0
141,E0,2023-12-05,20:15,Luton,Arsenal,3,4,A,1,2,...,1.82,2.14,1.85,2.07,1.81,premier_league,11,0,-1,-1
155,E0,2023-12-09,17:30,Aston Villa,Arsenal,1,0,H,1,0,...,2.03,1.91,2.07,1.86,2.02,premier_league,1,0,1,1
176,E0,2023-12-23,17:30,Liverpool,Arsenal,1,1,D,1,1,...,2.07,1.89,2.17,1.82,2.07,premier_league,10,0,0,0
194,E0,2023-12-31,14:00,Fulham,Arsenal,2,1,H,1,1,...,2.06,2.02,2.07,1.89,1.98,premier_league,9,0,1,0


In [7]:
arsenal_home_group_rolling = rolling_averages_for_window_sizes(arsenal_home_group,ROLLING_AVERAGES_COLUMNS,ROLLING_AVERAGES_WINDOW_SIZES,'H')
arsenal_home_group_rolling

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,H_R_HC_W_3,H_R_AC_W_3,H_R_HF_W_3,H_R_AF_W_3,H_R_HY_W_3,H_R_AY_W_3,H_R_HR_W_3,H_R_AR_W_3,H_R_FTR_code_W_3,H_R_HTR_code_W_3


In [8]:
arsenal_away_group_rolling = rolling_averages_for_window_sizes(arsenal_away_group,ROLLING_AVERAGES_COLUMNS,ROLLING_AVERAGES_WINDOW_SIZES,'A')
arsenal_away_group_rolling

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,A_R_HC_W_3,A_R_AC_W_3,A_R_HF_W_3,A_R_AF_W_3,A_R_HY_W_3,A_R_AY_W_3,A_R_HR_W_3,A_R_AR_W_3,A_R_FTR_code_W_3,A_R_HTR_code_W_3
18,E0,2023-08-21,20:00,Crystal Palace,Arsenal,0,1,A,0,0,...,5.666667,5.0,13.333333,12.666667,2.0,1.666667,0.0,0.0,-1.0,-1.0


## Rolling averages comparison home vs away team on whole season

In [9]:
data_home_rolling = data_home_grouped.apply(
    lambda x:
        rolling_averages_for_window_sizes(
            x,
            ROLLING_AVERAGES_COLUMNS,
            ROLLING_AVERAGES_WINDOW_SIZES,
            'H'
            )
        )
data_home_rolling

/var/folders/l3/bpd7p2510r9bclrfy43fqh0w0000gn/T/ipykernel_32205/609088817.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data_home_rolling = data_home_grouped.apply(


,,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,H_R_HC_W_3,H_R_AC_W_3,H_R_HF_W_3,H_R_AF_W_3,H_R_HY_W_3,H_R_AY_W_3,H_R_HR_W_3,H_R_AR_W_3,H_R_FTR_code_W_3,H_R_HTR_code_W_3
HomeTeam,,,,,,,,,,,,,,,,,,,,,,
Aston Villa,16,E0,2023-08-20,14:00,Aston Villa,Everton,4,0,H,2,0,...,7.333333,3.666667,12.333333,15.000000,1.666667,2.666667,0.333333,0.000000,0.000000,0.666667
Brentford,7,E0,2023-08-13,14:00,Brentford,Tottenham,2,2,D,2,2,...,3.333333,5.333333,11.666667,11.000000,2.333333,2.333333,0.000000,0.333333,0.333333,0.000000
Brighton,3,E0,2023-08-12,15:00,Brighton,Luton,4,1,H,1,0,...,9.333333,4.666667,11.666667,12.666667,2.333333,3.000000,0.000000,0.000000,0.333333,0.333333
Chelsea,8,E0,2023-08-13,16:30,Chelsea,Liverpool,1,1,D,1,1,...,6.000000,4.000000,13.333333,11.333333,3.000000,4.000000,0.333333,0.000000,0.000000,0.333333
Crystal Palace,18,E0,2023-08-21,20:00,Crystal Palace,Arsenal,0,1,A,0,0,...,5.666667,1.666667,10.333333,10.666667,2.333333,2.000000,0.000000,0.000000,1.000000,0.666667
Everton,4,E0,2023-08-12,15:00,Everton,Fulham,0,1,A,0,0,...,3.666667,5.666667,10.000000,11.333333,1.333333,2.000000,0.000000,0.000000,-1.000000,-0.333333
Fulham,11,E0,2023-08-19,15:00,Fulham,Brentford,0,3,A,0,1,...,9.333333,5.333333,7.333333,13.333333,2.333333,1.333333,0.000000,0.000000,0.000000,0.000000
Liverpool,12,E0,2023-08-19,15:00,Liverpool,Bournemouth,3,1,H,2,1,...,8.000000,2.333333,14.000000,14.666667,2.666667,3.666667,0.000000,0.000000,1.000000,0.333333
Man City,15,E0,2023-08-19,20:00,Man City,Newcastle,1,0,H,1,0,...,10.666667,2.333333,8.666667,12.000000,0.666667,2.333333,0.000000,0.000000,0.666667,-0.333333


In [10]:
data_away_rolling = data_away_grouped.apply(
    lambda x:
        rolling_averages_for_window_sizes(
            x,
            ROLLING_AVERAGES_COLUMNS,
            ROLLING_AVERAGES_WINDOW_SIZES,
            'A'
            )
        )
data_away_rolling

/var/folders/l3/bpd7p2510r9bclrfy43fqh0w0000gn/T/ipykernel_32205/2275609324.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data_away_rolling = data_away_grouped.apply(


,,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,A_R_HC_W_3,A_R_AC_W_3,A_R_HF_W_3,A_R_AF_W_3,A_R_HY_W_3,A_R_AY_W_3,A_R_HR_W_3,A_R_AR_W_3,A_R_FTR_code_W_3,A_R_HTR_code_W_3
AwayTeam,,,,,,,,,,,,,,,,,,,,,,
Arsenal,18,E0,2023-08-21,20:00,Crystal Palace,Arsenal,0,1,A,0,0,...,5.666667,5.000000,13.333333,12.666667,2.000000,1.666667,0.000000,0.000000,-1.000000,-1.000000
Aston Villa,6,E0,2023-08-12,17:30,Newcastle,Aston Villa,5,1,H,2,1,...,2.333333,10.333333,11.333333,9.666667,1.666667,2.000000,0.333333,0.000000,0.000000,0.333333
Bournemouth,12,E0,2023-08-19,15:00,Liverpool,Bournemouth,3,1,H,2,1,...,2.000000,10.666667,11.000000,13.333333,1.666667,2.666667,0.000000,0.000000,0.666667,0.333333
Brentford,11,E0,2023-08-19,15:00,Fulham,Brentford,0,3,A,0,1,...,6.333333,3.666667,12.666667,11.333333,1.333333,2.333333,0.000000,0.000000,0.333333,-0.333333
Brighton,13,E0,2023-08-19,15:00,Wolves,Brighton,1,4,A,0,1,...,7.000000,6.666667,10.000000,10.666667,1.666667,3.000000,0.333333,0.000000,0.333333,-0.333333
Chelsea,17,E0,2023-08-20,16:30,West Ham,Chelsea,3,1,H,1,1,...,4.333333,4.000000,12.333333,11.666667,3.000000,3.000000,0.000000,0.000000,0.333333,0.666667
Crystal Palace,5,E0,2023-08-12,15:00,Sheffield United,Crystal Palace,0,1,A,0,0,...,6.666667,4.666667,10.000000,9.666667,3.333333,2.666667,0.000000,0.000000,0.333333,0.000000
Everton,16,E0,2023-08-20,14:00,Aston Villa,Everton,4,0,H,2,0,...,7.000000,6.333333,12.000000,12.000000,0.666667,2.333333,0.000000,0.000000,0.666667,0.666667
Fulham,4,E0,2023-08-12,15:00,Everton,Fulham,0,1,A,0,0,...,5.000000,3.333333,9.333333,10.666667,1.666667,3.000000,0.000000,0.333333,0.333333,0.000000


## Rolling averages first groped by home team and then by away team    

In [11]:
data_home_rolling

,,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,H_R_HC_W_3,H_R_AC_W_3,H_R_HF_W_3,H_R_AF_W_3,H_R_HY_W_3,H_R_AY_W_3,H_R_HR_W_3,H_R_AR_W_3,H_R_FTR_code_W_3,H_R_HTR_code_W_3
HomeTeam,,,,,,,,,,,,,,,,,,,,,,
Aston Villa,16,E0,2023-08-20,14:00,Aston Villa,Everton,4,0,H,2,0,...,7.333333,3.666667,12.333333,15.000000,1.666667,2.666667,0.333333,0.000000,0.000000,0.666667
Brentford,7,E0,2023-08-13,14:00,Brentford,Tottenham,2,2,D,2,2,...,3.333333,5.333333,11.666667,11.000000,2.333333,2.333333,0.000000,0.333333,0.333333,0.000000
Brighton,3,E0,2023-08-12,15:00,Brighton,Luton,4,1,H,1,0,...,9.333333,4.666667,11.666667,12.666667,2.333333,3.000000,0.000000,0.000000,0.333333,0.333333
Chelsea,8,E0,2023-08-13,16:30,Chelsea,Liverpool,1,1,D,1,1,...,6.000000,4.000000,13.333333,11.333333,3.000000,4.000000,0.333333,0.000000,0.000000,0.333333
Crystal Palace,18,E0,2023-08-21,20:00,Crystal Palace,Arsenal,0,1,A,0,0,...,5.666667,1.666667,10.333333,10.666667,2.333333,2.000000,0.000000,0.000000,1.000000,0.666667
Everton,4,E0,2023-08-12,15:00,Everton,Fulham,0,1,A,0,0,...,3.666667,5.666667,10.000000,11.333333,1.333333,2.000000,0.000000,0.000000,-1.000000,-0.333333
Fulham,11,E0,2023-08-19,15:00,Fulham,Brentford,0,3,A,0,1,...,9.333333,5.333333,7.333333,13.333333,2.333333,1.333333,0.000000,0.000000,0.000000,0.000000
Liverpool,12,E0,2023-08-19,15:00,Liverpool,Bournemouth,3,1,H,2,1,...,8.000000,2.333333,14.000000,14.666667,2.666667,3.666667,0.000000,0.000000,1.000000,0.333333
Man City,15,E0,2023-08-19,20:00,Man City,Newcastle,1,0,H,1,0,...,10.666667,2.333333,8.666667,12.000000,0.666667,2.333333,0.000000,0.000000,0.666667,-0.333333


In [12]:
home_rolling_away_group = data_home_rolling.groupby('AwayTeam')

In [13]:
home_rolling_away_group_arsenal = home_rolling_away_group.get_group('Arsenal')
home_rolling_away_group_arsenal = home_rolling_away_group_arsenal.sort_values("Date")
home_rolling_away_group_arsenal


,,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,H_R_HC_W_3,H_R_AC_W_3,H_R_HF_W_3,H_R_AF_W_3,H_R_HY_W_3,H_R_AY_W_3,H_R_HR_W_3,H_R_AR_W_3,H_R_FTR_code_W_3,H_R_HTR_code_W_3
HomeTeam,,,,,,,,,,,,,,,,,,,,,,
Crystal Palace,18,E0,2023-08-21,20:00,Crystal Palace,Arsenal,0,1,A,0,0,...,5.666667,1.666667,10.333333,10.666667,2.333333,2.0,0.0,0.0,1.0,0.666667


In [14]:

home_rolling_away_group_arsenal_rolling = rolling_averages_for_window_sizes(
    home_rolling_away_group_arsenal,
    ROLLING_AVERAGES_COLUMNS,
    ROLLING_AVERAGES_WINDOW_SIZES,
    'A'
    )
home_rolling_away_group_arsenal_rolling

ValueError: cannot join with no overlapping index names

In [ ]:
home_rolling_away_rolling = home_rolling_away_group.apply(
    lambda x:
        rolling_averages_for_window_sizes(
            x,
            ROLLING_AVERAGES_COLUMNS,
            ROLLING_AVERAGES_WINDOW_SIZES,
            'A'
            )
        )
home_rolling_away_rolling

/var/folders/l3/bpd7p2510r9bclrfy43fqh0w0000gn/T/ipykernel_404/3629782419.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  home_rolling_away_rolling = home_rolling_away_group.apply(


Div       Date   Time       HomeTeam AwayTeam  \
AwayTeam HomeTeam                                                          
Arsenal  Aston Villa   155  E0 2023-12-09  17:30    Aston Villa  Arsenal   
         Brentford     126  E0 2023-11-25  17:30      Brentford  Arsenal   
         Brighton      309  E0 2024-04-06  17:30       Brighton  Arsenal   
         Burnley       239  E0 2024-02-17  15:00        Burnley  Arsenal   
         Fulham        194  E0 2023-12-31  14:00         Fulham  Arsenal   
...                         ..        ...    ...            ...      ...   
Wolves   Man City      352  E0 2024-05-04  17:30       Man City   Wolves   
         Newcastle     261  E0 2024-03-02  15:00      Newcastle   Wolves   
         Nott'm Forest 317  E0 2024-04-13  15:00  Nott'm Forest   Wolves   
         Tottenham     243  E0 2024-02-17  15:00      Tottenham   Wolves   
         West Ham      167  E0 2023-12-17  14:00       West Ham   Wolves   

                            FTHG  FTAG FTR  HTHG  HTAG  ... A_R_HC_W_3  \
AwayTeam HomeTeam                                       ...              
Arsenal  Aston Villa   155     1     0   H     1     0  ...   1.333333   
         Brentford     126     0     1   A     0     0  ...   2.666667   
         Brighton      309     0     3   A     0     1  ...   3.666667   
         Burnley       239     0     5   A     0     2  ...   3.000000   
         Fulham        194     2     1   H     1     1  ...   3.333333   
...                          ...   ...  ..   ...   ...  ...        ...   
Wolves   Man City      352     5     1   H     3     0  ...   7.333333   
         Newcastle     261     3     0   H     2     0  ...   9.333333   
         Nott'm Forest 317     2     2   D     1     1  ...   5.000000   
         Tottenham     243     1     2   A     0     1  ...   8.666667   
         West Ham      167     3     0   H     2     0  ...   4.666667   

                           A_R_AC_W_3  A_R_HF_W_3  A_R_AF_W_3  A_R_HY_W_3  \
AwayTeam HomeTeam                                                           
Arsenal  Aston Villa   155   9.000000   12.333333    7.666667    2.666667   
         Brentford     126   8.000000   10.666667   11.000000    2.666667   
         Brighton      309   5.666667   10.333333   10.666667    1.000000   
         Burnley       239   7.000000   11.333333    9.333333    2.666667   
         Fulham        194   5.333333   14.333333   10.666667    2.666667   
...                               ...         ...         ...         ...   
Wolves   Man City      352   3.000000   12.000000   13.666667    1.666667   
         Newcastle     261   2.333333   11.000000   12.333333    1.333333   
         Nott'm Forest 317   4.333333    9.666667   14.000000    1.000000   
         Tottenham     243   1.666667   10.000000   12.666667    1.666667   
         West Ham      167   2.000000   12.000000   11.000000    1.666667   

                            A_R_AY_W_3  A_R_HR_W_3  A_R_AR_W_3  \
AwayTeam HomeTeam                                                
Arsenal  Aston Villa   155    1.000000         0.0         0.0   
         Brentford     126    1.666667         0.0         0.0   
         Brighton      309    1.000000         0.0         0.0   
         Burnley       239    0.666667         0.0         0.0   
         Fulham        194    2.666667         0.0         0.0   
...                                ...         ...         ...   
Wolves   Man City      352    3.000000         0.0         0.0   
         Newcastle     261    2.333333         0.0         0.0   
         Nott'm Forest 317    2.333333         0.0         0.0   
         Tottenham     243    2.333333         0.0         0.0   
         West Ham      167    2.333333         0.0         0.0   

                            A_R_FTR_code_W_3  A_R_HTR_code_W_3  
AwayTeam HomeTeam                                               
Arsenal  Aston Villa   155         -0.333333         -0.333333  
         Brentford 